## Functions

### Never Unpack More Than Three Variables When Functions Return Multiple Values

python支持函数多返回值，支持unpacking。

导致我也偶尔写过这样的代码(

`a, b, c, d, e, f, g, h, i = func()`

这样的代码有两个明显的问题

- 顺序很容易出错，需要对照检查，出错还很难发现。。
- 可读性较差（如果代码要求pylint检查的话，行长度限制会使得这个语句七零八落）

因此

- 返回值不要超过3个
- 否则， return a small class or namedtuple instance来替代
> transformers中就使用了这种写法


### Prefer Raising Exceptions to Returning None

return None导致问题的最常见情景是：

主过程中用`if not result`，而不是`if result is not None`来判断。而函数有正确的返回值0, 或是空字符串。

一种常见的解决方式是返回一个tuple, `(flag, result)`，但调用者很容易就会忽视掉flag, 写起来也显得冗余

因此推荐的写法就是在函数中raise, 由调用者来handle.
- 复杂的API推荐派生Exception, 实现自己的异常处理类。

In [1]:
# 一个完整的例子，包括type annotation和docstring
def careful_divide(a: float, b: float) -> float:
    """Divides a by b.

    Raises:
    ValueError: When the inputs cannot be divided.
    """
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

### Know How Closures Interact with Variable Scope

嵌套函数定义我们经常使用。

内部定义的函数，可以直接访问外层函数的变量。

但如果内部定义函数对一个外层函数含有的变量进行赋值，实际行为是在内部函数的作用域内重新创建了一个新的同名变量，而不是修改外层函数变量的值。

为此要使用nonlocal关键字 ("complementary to the `global` statement")

In [2]:
def sort_priority(numbers, group):
    found = False
    def helper(x):
        nonlocal found # 否则return的found只会是False
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = [2, 3, 5, 7]
found = sort_priority(numbers, group)
print('found: ', found)
print('numbers = ', numbers)

found:  True
numbers =  [2, 3, 5, 7, 1, 4, 6, 8]


In [3]:
# 注意，nonlocal是可以用class的成员变量轻松实现的
# 因此当nonlocal比较复杂的时候，不妨用class来替代

class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)
sorter = Sorter(group)
numbers.sort(key=sorter)
print('found: ', sorter.found)
print('numbers = ', numbers)

found:  True
numbers =  [2, 3, 5, 7, 1, 4, 6, 8]


### Provide Optional Behavior with Keyword Arguments

除了普遍支持的position argument外，python还支持keyword argument

keyword argument的好处主要有以下三点
- 增强了函数调用的可读性
- 若函数有多个optional argument，调用时可以通过keyword argument直接设置其中的一个或几个。
- 扩展函数参数时不需要改之前的调用。(新增的参数提供默认值,即optional参数，新的调用用keyword argument设置新增参数)

In [4]:
# keyword argument的形式有多种

def func(number, divisor):
    return number % divisor

print(func(number=20, divisor=7))

kwargs = {'number': 20, 'divisor': 7}
print(func(**kwargs))

kwargs = {'divisor': 7}
print(func(number=20, **kwargs))

def print_parameters(**kwargs):
    for k, v in kwargs.items():
        print(f'{k} = {v}')

print_parameters(alpha=1.5, beta=9)


6
6
6
alpha = 1.5
beta = 9


### Use `None` and Docstrings to Specify Dynamic Default Arguments

这个有点坑。。。

"A default argument value is evaluated only once during function definition at module load time, which usually happens when a program starts up. After the module containing this code is loaded, default arguemnt will never be evaluated again"

即参数的默认值会在load模块时赋值一次，之后就不再重复了。

如果默认的value是static的，比如一个int, 那不会导致错误。

但如果默认的value是dynamic(比如`[]`, `{}`, `datetime.now()`)，这个特性就会导致unexpected behavior的发生

In [5]:
from time import sleep
from datetime import datetime

def log(message, when=datetime.now()): # 这里的now只调用一次，导致错误
    print(f'{when}: {message}')
    
log("Hi there!")
sleep(0.1)
log('Hello again!')

2021-02-21 00:40:28.591086: Hi there!
2021-02-21 00:40:28.591086: Hello again!


In [6]:
import json

def decode(data, default = {}): # 同一个default在传递
    try:
        return json.loads(data)
    except ValueError:
        return default

foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5, 'meep': 1}
Bar: {'stuff': 5, 'meep': 1}


In [7]:
# 因此应该用None替代dynamic value作为default value

from typing import Optional

def log_typed(message: str,
    when: Optional[datetime]=None) -> None:
    """Log a message with a timestamp.
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

log_typed("Hi there!")
sleep(0.1)
log_typed('Hello again!')

2021-02-21 00:40:28.725004: Hi there!
2021-02-21 00:40:28.825524: Hello again!


### Enforce Clarity with Keyword-Only and Positional-Only Arguments

对于没有强制要求的keyword argument, 调用者依然可能用position argument的方式调用，可读性没有得到改善。

而对于没有强制要求的position argument, 调用者反而可能用keyword argument的方式调用，更改参数名后原来的调用就会挂掉。

为了让函数实现者的苦心不被辜负，python 3.8引入了positional-only argument, 与之前已经支持的keyword-only argument一齐，解决了这个问题

- keyword-only: *将其后的参数设为keyword-only
- positional-only: /将之前的参数设为positional-only
- *和/之间的参数不设限制 (python对于parameter的默认设置)

In [8]:
def func(number, divisor, /, n, *, flag=False):
    print('call')

# func(number=1, 2) 报错
# func(1, 2, True) 报错
func(1, 2, 3, flag=True)
func(1, 2, n=3, flag=True)

call
call


### Define Function Decorators with `functool.wrap`

装饰器是很常用的feature.

这里主要是想说明,"Using decorators can cause **strange behaviors** in tools that do introspection, such as debuggers"

因为返回的函数是装饰过的函数，不再是原函数，`__name__`，`docstring`, `__module__`, `__annotations__`等属性都改变了。

为此，可以使用functools中的wraps,

>"When you apply it to the wrapper, it copies all of the important metadata about the inner function to the outer function"

In [9]:
def trace(func):
    def wrapper(*args, **kwargs): # positional argument和keyword argument
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

fibonacci(4)
help(fibonacci)
print(fibonacci.__name__)

fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci((4,), {}) -> 3
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)

wrapper


In [10]:
from functools import wraps

def trace2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

@trace2
def fibonacci2(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

fibonacci2(4)
help(fibonacci2)
print(fibonacci2.__name__)

fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci2((4,), {}) -> 3
Help on function fibonacci2 in module __main__:

fibonacci2(n)
    Return the n-th Fibonacci number

fibonacci2
